## EDA on ACLU Bill Data

In [36]:
# IMPORT PACKAGES
import pandas as pd
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt

In [29]:
# LOAD IN THE DATA
data = pd.read_csv("../modified_data/merged_bill_data.csv")
# Drop the unnamed column
data = data.drop(data.columns[0], axis = 1)
# Get the number of characters in each bill
data["number_characters"] = data["text"].str.len()

### Tokenize Option 1: Tokenizing with NLTK

In [33]:
data["nltk_tokens"] = [word_tokenize(text) for text in data["text"]]
# Get the number of tokens with NLTK in each bill
data["number_nltk_tokens"] = data["nltk_tokens"].str.len()

In [39]:
# Get the number of bills with less than or equal to 512 tokens
len(data[(data["number_nltk_tokens"] < 512)])

86